In [1]:
import os
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:Path
    local_data_file:Path                                   ## Name should always same
    unzip_dir:Path
    ## Data class we don't need to use self variable

In [4]:
from src.ds_project.constants import *
from src.ds_project.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        print(self.config.artifacts_root)

        create_directories(self.config.artifacts_root)

    def get_data_ingestion_path(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories(config.root_dir)

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config



In [6]:
import os
import urllib.request as request
from src.ds_project import logger
import zipfile
import requests

In [7]:
## Components

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    ## Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):  # Check if the file already exists
            try:
                # Send an HTTP GET request to download the file
                response = requests.get(self.config.source_URL)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Write the content to the local file
                    with open(self.config.local_data_file, 'wb') as file:
                        file.write(response.content)
                    logger.info(f"{self.config.local_data_file} downloaded successfully.")
                else:
                    logger.error(f"Failed to download the file. Status code: {response.status_code}")
            except Exception as e:
                logger.error(f"An error occurred while downloading the file: {e}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists. Skipping download.")
    ## extract the zip file
    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

            

In [8]:
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project'

In [9]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_path()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip()
except Exception as e:
    raise e


<_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'>
{'artifacts_root': ['artifacts'], 'data_ingestion': {'root_dir': ['artifacts/data_ingestion'], 'source_URL': 'https://raw.githubusercontent.com/dushyantverma22/data_set/main/WineQT.csv', 'local_data_file': 'artifacts/data_ingestion/WineQT.csv', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-10-21 19:00:25,823: INFO: common: yaml file: config\config.yaml loaded successfully]
<_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'>
{'key': 'value'}
[2024-10-21 19:00:25,824: INFO: common: yaml file: params.yaml loaded successfully]
<_io.TextIOWrapper name='schema.yaml' mode='r' encoding='utf-8'>
{'key1': 'value1'}
[2024-10-21 19:00:25,825: INFO: common: yaml file: schema.yaml loaded successfully]
['artifacts']
[2024-10-21 19:00:25,825: INFO: common: Directory created: artifacts]
[2024-10-21 19:00:25,825: INFO: common: Directory created: artifacts/data_ingestion]
[2024-10-21 19:00:26,177: INFO: 3634282551: ar